In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import requests
import pymongo

매장 정보를 얻어오기 위해 시도코드와 시도코드에 따른 구군코드 얻어오기 위해 mongodb에 저장된 데이터를 얻어온다.

In [2]:
mongoConnection = pymongo.MongoClient('mongodb://localhost:27017')
starbucks = mongoConnection['starbucks']
gugunAll = starbucks['gugunAll']
gugunList = gugunAll.find({}, {'_id': 0}) # starbucks> db.gugunAll.find({}, {'_id': 0})

s_list = set()
for g in gugunList:
    print(g)
    s_list.add(g['sido_nm'])
print(s_list)

{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0101', 'gugun_nm': '강남구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0120', 'gugun_nm': '강동구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0102', 'gugun_nm': '강북구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0103', 'gugun_nm': '강서구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0104', 'gugun_nm': '관악구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0105', 'gugun_nm': '광진구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0121', 'gugun_nm': '구로구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0106', 'gugun_nm': '금천구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0107', 'gugun_nm': '노원구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0108', 'gugun_nm': '도봉구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0122', 'gugun_nm': '동대문구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0109', 'gugun_nm': '동작구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0110', 'gugun_nm': '마포구'}
{'sido_cd': '01', 'sido_nm': '서울', 'g

시도이름과 구군이름을 입력받아 매장 정보를 얻어와서 mongodb에 저장할 도큐먼트를 만들어서 저장한다.

In [3]:
# 시도이름과 구군이름을 입력받아 입력받은 지역의 매장 정보를 크롤링한다.
print('시도이름: ', end='')
for s in s_list:
    print('[{}]'.format(s), end=' ')
print()
sidoName = input('매장 정보를 검색하려는 시도이름을 입력하세요(예: 서울): ').strip()

g_list = gugunAll.find({'sido_nm': sidoName}, {'_id': 0}) # starbucks> db.gugunAll.find({'sido_nm': '서울'}, {'_id': 0})
print('구군이름: ', end='')
for g in g_list:
    print('[{}]'.format(g['gugun_nm']), end=' ')
print()
gugunName = input('매장 정보를 검색하려는 구군이름을 입력하세요(예: 종로구): ').strip()
# print(sidoName)
# print(gugunName)
'''
gugunList = gugunAll.find(
    {
        '$and': [
            {'sido_nm':
                {'$eq': sidoName}
            },
            {'gugun_nm':
                {'$regex': '^' + gugunName}
            }
        ]
    }, 
    {'_id': 0, 'gugun_cd': 1}
)
'''
gugunList = gugunAll.find({'$and': [ {'sido_nm': {'$eq': sidoName} }, {'gugun_nm': {'$regex': '^' + gugunName} } ]}, {'_id': 0})
gugunCode = []
for gugun in gugunList:
    # print(gugun, gugun['gugun_cd'])
    gugunCode.append(gugun)
# print(gugunCode)

storegugun = starbucks['storegugun'] # starbucks> db.createCollection('storegugun')
storegugun.delete_many({})
for gugun in gugunCode:
    # print(gugun)
    request = requests.post(
        'https://www.starbucks.co.kr/store/getStore.do?r=ZSC0MF0FUT',
        data={
            'ins_lat': 37.56682,
            'ins_lng': 126.97865,
            'p_sido_cd': gugun['sido_cd'],
            'p_gugun_cd': gugun['gugun_cd'],
            'in_biz_cd': '',
            'iend': 2000,
            'set_date': '',
        }).json()
    print('{}({}) {}({})의 스타벅스 매장: {}개'.format(gugun['sido_nm'], gugun['sido_cd'], gugun['gugun_nm'], gugun['gugun_cd'], len(request['list'])))
    # print(request['list'][0])
    
    storeList = []
    for req in request['list']:
        dic = {
            'sido_code': req['sido_code'], # 시도코드
            'sido_name': req['sido_name'], # 시도이름
            'gugun_code': req['gugun_code'], # 구군코드
            'gugun_name': req['gugun_name'], # 구군이름
            's_name': req['s_name'], # 지점명
            'tel': req['tel'], # 전화번호
            'addr': req['addr'], # 지번주소
            'doro_address': req['doro_address'], # 도로명주소
            'lat': req['lat'], # 위도
            'lot': req['lot'] # 경도
        }
        # print(dic)
        storeList.append(dic)

    for store in storeList:
        storegugun.insert_one(store)

시도이름: [경기] [광주] [대구] [대전] [경북] [서울] [전남] [충남] [세종] [전북] [충북] [인천] [강원] [제주] [부산] [경남] [울산] 
매장 정보를 검색하려는 시도이름을 입력하세요(예: 서울): 서울
구군이름: [강남구] [강동구] [강북구] [강서구] [관악구] [광진구] [구로구] [금천구] [노원구] [도봉구] [동대문구] [동작구] [마포구] [서대문구] [서초구] [성동구] [성북구] [송파구] [양천구] [영등포구] [용산구] [은평구] [종로구] [중구] [중랑구] 
매장 정보를 검색하려는 구군이름을 입력하세요(예: 종로구): 강남구
서울(01) 강남구(0101)의 스타벅스 매장: 91개


In [8]:
mongoConnection = pymongo.MongoClient('mongodb://localhost:27017')
starbucks = mongoConnection['starbucks']
storegugun = starbucks['storegugun']
gugunList = storegugun.find({}, {'_id': 0})
for g in gugunList:
    print(g)

{'sido_code': '01', 'sido_name': '서울', 'gugun_code': '0101', 'gugun_name': '강남구', 's_name': '강남세곡DT', 'tel': '1522-3232', 'addr': '서울특별시 강남구 세곡동 21-3, 21-1, 21-10(일부)', 'doro_address': '서울특별시 강남구 헌릉로 727 (세곡동)', 'lat': '37.466993809277', 'lot': '127.118330806807'}
{'sido_code': '01', 'sido_name': '서울', 'gugun_code': '0101', 'gugun_name': '강남구', 's_name': '역삼이마트', 'tel': '1522-3232', 'addr': '서울특별시 강남구 역삼동 755 한솔필리아', 'doro_address': '서울특별시 강남구 역삼로 310 (역삼동)', 'lat': '37.499367', 'lot': '127.048425'}
{'sido_code': '01', 'sido_name': '서울', 'gugun_code': '0101', 'gugun_name': '강남구', 's_name': '압구정현대아파트', 'tel': '1522-3232', 'addr': '서울특별시 강남구 신사동 574-4', 'doro_address': '서울특별시 강남구 논현로175길 38 (신사동)', 'lat': '37.52521971236768', 'lot': '127.0256601858147'}
{'sido_code': '01', 'sido_name': '서울', 'gugun_code': '0101', 'gugun_name': '강남구', 's_name': '청담역', 'tel': '1522-3232', 'addr': '서울특별시 강남구 청담동 47-2', 'doro_address': '서울특별시 강남구 삼성로 709 (청담동)', 'lat': '37.5194407323478', 'lot': '127.0495526

시도이름을 입력받아 매장 정보를 얻어와서 mongodb에 저장할 도큐먼트를 만들어서 저장한다.

In [5]:
# 시도이름을 입력받아 입력받은 지역의 매장 정보를 크롤링한다.
print('시도이름: ', end='')
for s in s_list:
    print('[{}]'.format(s), end=' ')
print()
sidoName = input('매장 정보를 검색하려는 시도이름을 입력하세요(예: 서울): ').strip()

starbucks = mongoConnection['starbucks']
sido = starbucks['sido']
sidoCode = sido.find({'sido_nm': sidoName}, {'_id': 0}).next()

request = requests.post(
    'https://www.starbucks.co.kr/store/getStore.do?r=ZSC0MF0FUT',
    data={
        'ins_lat': 37.56682, 'ins_lng': 126.97865, 'p_sido_cd': sidoCode['sido_cd'], 'p_gugun_cd': '',
        'in_biz_cd': '', 'iend': 2000, 'set_date': '',
    }).json()
print('{}({})의 스타벅스 매장: {}개'.format(sidoCode['sido_nm'], sidoCode['sido_cd'], len(request['list'])))

storeList = []
for req in request['list']:
    dic = {'sido_code': req['sido_code'], 'sido_name': req['sido_name'], 'gugun_code': req['gugun_code'], 
           'gugun_name': req['gugun_name'], 's_name': req['s_name'], 'tel': req['tel'], 'addr': req['addr'],
           'doro_address': req['doro_address'], 'lat': req['lat'], 'lot': req['lot']
    }
    storeList.append(dic)

storesido = starbucks['storesido']
storesido.delete_many({})
for store in storeList:
    storesido.insert_one(store)

시도이름: [경기] [광주] [대구] [대전] [경북] [서울] [전남] [충남] [세종] [전북] [충북] [인천] [강원] [제주] [부산] [경남] [울산] 
매장 정보를 검색하려는 시도이름을 입력하세요(예: 서울): 경기
경기(08)의 스타벅스 매장: 469개


In [9]:
mongoConnection = pymongo.MongoClient('mongodb://localhost:27017')
starbucks = mongoConnection['starbucks']
storesido = starbucks['storesido']
gugunList = storesido.find({}, {'_id': 0})
for g in gugunList:
    print(g)

{'sido_code': '08', 'sido_name': '경기', 'gugun_code': '0801', 'gugun_name': '가평군', 's_name': '리버사이드대성리DT', 'tel': '1522-3232', 'addr': '경기도 가평군 청평면 대성리 344-22', 'doro_address': '경기도 가평군 청평면 경춘로 258 ', 'lat': '37.69825440707157', 'lot': '127.3852165854917'}
{'sido_code': '08', 'sido_name': '경기', 'gugun_code': '0801', 'gugun_name': '가평군', 's_name': '가평설악IC DT', 'tel': '1522-3232', 'addr': '경기도 가평군 설악면 신천리 554-5, 6, 9, 10', 'doro_address': '경기도 가평군 설악면 신선봉로 14 ', 'lat': '37.67905252127808', 'lot': '127.48638178238548'}
{'sido_code': '08', 'sido_name': '경기', 'gugun_code': '0801', 'gugun_name': '가평군', 's_name': '남이섬', 'tel': '1522-3232', 'addr': '경기도 가평군 가평읍 달전리 113-4, 11', 'doro_address': '경기도 가평군 가평읍 북한강변로 1054 북한강변로 1054', 'lat': '37.806624', 'lot': '127.52527'}
{'sido_code': '08', 'sido_name': '경기', 'gugun_code': '0802', 'gugun_name': '고양시 일산동구', 's_name': '소노캄고양', 'tel': '1522-3232', 'addr': '경기도 고양시 일산동구 장항동 1755 소노캄고양웨스트타워', 'doro_address': '경기도 고양시 일산동구 태극로 18 (장항동)', 'lat': '37.6628

전국 매장 정보를 얻어와서 mongodb에 저장할 도큐먼트를 만들어서 저장한다.

In [7]:
request = requests.post(
    'https://www.starbucks.co.kr/store/getStore.do?r=ZSC0MF0FUT',
    data={
        'ins_lat': 37.56682, 'ins_lng': 126.97865, 'p_sido_cd': '', 'p_gugun_cd': '',
        'in_biz_cd': '', 'iend': 2000, 'set_date': '',
    }).json()
print('전국의 스타벅스 매장: {}개'.format(len(request['list'])))

storeList = []
for req in request['list']:
    dic = {'sido_code': req['sido_code'], 'sido_name': req['sido_name'], 'gugun_code': req['gugun_code'], 
           'gugun_name': req['gugun_name'], 's_name': req['s_name'], 'tel': req['tel'], 'addr': req['addr'],
           'doro_address': req['doro_address'], 'lat': req['lat'], 'lot': req['lot']
    }
    storeList.append(dic)

storeall = starbucks['storeall']
storeall.delete_many({})
for store in storeList:
    storeall.insert_one(store)

전국의 스타벅스 매장: 1923개


In [10]:
mongoConnection = pymongo.MongoClient('mongodb://localhost:27017')
starbucks = mongoConnection['starbucks']
storeall = starbucks['storeall']
gugunList = storeall.find({}, {'_id': 0})
for g in gugunList:
    print(g)

{'sido_code': '03', 'sido_name': '대구', 'gugun_code': '0305', 'gugun_name': '수성구', 's_name': '대구수성호수R', 'tel': '1522-3232', 'addr': '대구광역시 수성구 두산동 911-5', 'doro_address': '대구광역시 수성구 용학로 56 (두산동)', 'lat': '35.826126', 'lot': '128.615912'}
{'sido_code': '05', 'sido_name': '부산', 'gugun_code': '0516', 'gugun_name': '해운대구', 's_name': '센텀시티역', 'tel': '1522-3232', 'addr': '부산광역시 해운대구 우동 1498 트럼프월드센텀 1층', 'doro_address': '부산광역시 해운대구 센텀동로 9, 트럼프월드센텀아파트 1층 (우동)', 'lat': '35.169925', 'lot': '129.132271'}
{'sido_code': '05', 'sido_name': '부산', 'gugun_code': '0508', 'gugun_name': '연제구', 's_name': '연산중앙로', 'tel': '1522-3232', 'addr': '부산광역시 연제구 연산동 1242-12, 30', 'doro_address': '부산광역시 연제구 중앙대로 1091 (연산동)', 'lat': '35.185469', 'lot': '129.08067'}
{'sido_code': '01', 'sido_name': '서울', 'gugun_code': '0114', 'gugun_name': '송파구', 's_name': '송파마천사거리DT', 'tel': '1522-3232', 'addr': '서울특별시 송파구 마천동 43-1', 'doro_address': '서울특별시 송파구 거마로 78 (마천동)', 'lat': '37.499829', 'lot': '127.147672'}
{'sido_code': '05', '